<h1> About this notebook </h1>

This notebook has been created to demo capabilities of Databricks notebooks and Azure Cognitive Services entity recognition API.

Following Azure services have been used in this demo:
1. Azure Data Lake Storage (ADLS Gen2) - used for storing files that have to be processed for PII detection.
2. Azure Databricks - one stop shop to run our program code; transform data; interact with other Azure services to deliver a complete end to end solution.
3. Azure SQL DB - used as a metadata store where Databricks writes information like job start/end time; file name that is being processed etc.
4. Azure Cosmos DB - store output of Azure Cognitive Services in case PII is detected. Where no PII is detected, the output is ignored. 
5. Azure Cognitive Services / Text Analytics API -  (Named Entity Recognition) is used for detecting PII.
</br>




**To run this notebook in your subscription, please replace text in <> with your account keys/credentials. **

<h4> Install Python packages required for this notebook </h4>


Install packages for Python required to run this notebook. 

** You only need to run this step once when you run this notebook for the first time. **

In [3]:
# Install packages
# Note - installation of packages can be automated 

dbutils.library.installPyPI("requests")
dbutils.library.installPyPI("simpleJson")
dbutils.library.installPyPI("pyodbc")
dbutils.library.installPyPI("azure.cosmos")
dbutils.library.installPyPI("requests")

PyPI package azure.cosmos has been installed already. The previous installed package is PyPI:(azure.cosmos)-(empty)-(empty)-(empty). To resolve this issue detach and retach the notebook to create a new environment or rename the package.
Out[20]: False

<h4> Install ODBC drivers for Microsoft SQL Server on Databricks cluster </h4>

We are using PYODBC package to connect to Azure SQL DB. This step installs the SQL Server driver.

** You only need to run this step once when you run this notebook for the first time. **

In [5]:
%sh

curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
curl https://packages.microsoft.com/config/ubuntu/16.04/prod.list > /etc/apt/sources.list.d/mssql-release.list 
apt-get update
ACCEPT_EULA=Y apt-get install msodbcsql17
apt-get -y install unixodbc-dev

% Total % Received % Xferd Average Speed Time Time Time Current
 Dload Upload Total Spent Left Speed

 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0
100 983 100 983 0 0 1226 0 --:--:-- --:--:-- --:--:-- 1225
100 983 100 983 0 0 1226 0 --:--:-- --:--:-- --:--:-- 1225
OK
 % Total % Received % Xferd Average Speed Time Time Time Current
 Dload Upload Total Spent Left Speed

 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0
100 79 100 79 0 0 144 0 --:--:-- --:--:-- --:--:-- 144
Get:1 https://packages.microsoft.com/ubuntu/16.04/prod xenial InRelease [4,003 B]
Get:2 https://cran.rstudio.com/bin/linux/ubuntu xenial-cran35// InRelease [3,625 B]
Get:3 http://security.ubuntu.com/ubuntu xenial-security InRelease [109 kB]
Get:4 https://packages.microsoft.com/ubuntu/16.04/prod xenial/main amd64 Packages [120 kB]
Hit:5 http://archive.ubuntu.com/ubuntu xenial InRelease
Get:6 https://cran.rstudio.com/bin/linux/ubuntu xenial-cran35// Packages [78.7 kB]
Get:7 http://archive.ubuntu.com/ubuntu xenial-updates InRelease [109 kB]
Get:8 http://security.ubuntu.com/ubuntu xenial-security/main amd64 Packages [1,003 kB]
Get:9 http://archive.ubuntu.com/ubuntu xenial-backports InRelease [107 kB]
Get:10 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 Packages [1,381 kB]
Get:11 http://security.ubuntu.com/ubuntu xenial-security/universe amd64 Packages [591 kB]
Get:12 http://archive.ubuntu.com/ubuntu xenial-updates/universe amd64 Packages [995 kB]
Get:13 http://security.ubuntu.com/ubuntu xenial-security/multiverse amd64 Packages [6,283 B]
Get:14 http://archive.ubuntu.com/ubuntu xenial-updates/multiverse amd64 Packages [19.3 kB]
Fetched 4,528 kB in 9s (502 kB/s)
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
The following package was automatically installed and is no longer required:
 libgnutls-openssl27
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
 msodbcsql17
0 upgraded, 1 newly installed, 0 to remove and 60 not upgraded.
Need to get 750 kB of archives.
After this operation, 0 B of additional disk space will be used.
Get:1 https://packages.microsoft.com/ubuntu/16.04/prod xenial/main amd64 msodbcsql17 amd64 17.4.2.1-1 [750 kB]
debconf: delaying package configuration, since apt-utils is not installed
Fetched 750 kB in 0s (751 kB/s)
Selecting previously unselected package msodbcsql17.
(Reading database ... 
(Reading database ... 5%
(Reading database ... 10%
(Reading database ... 15%
(Reading database ... 20%
(Reading database ... 25%
(Reading database ... 30%
(Reading database ... 35%
(Reading database ... 40%
(Reading database ... 45%
(Reading database ... 50%
(Reading database ... 55%
(Reading database ... 60%
(Reading database ... 65%
(Reading database ... 70%
(Reading database ... 75%
(Reading database ... 80%
(Reading database ... 85%
(Reading database ... 90%
(Reading database ... 95%
(Reading database ... 100%
(Reading database ... 61911 files and directories currently installed.)
Preparing to unpack .../msodbcsql17_17.4.2.1-1_amd64.deb ...
debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
Unpacking msodbcsql17 (17.4.2.1-1) ...
Setting up msodbcsql17 (17.4.2.1-1) ...
Reading package lists...
Building dependency tree...
Reading state information...
The following package was automatically installed and is no longer required:
 libgnutls-openssl27
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
 libodbc1 odbcinst odbcinst1debian2 unixodbc
Suggested packages:
 unixodbc-bin
The following packages will be upgraded:
 libodbc1 odbcinst odbcinst1debian2 unixodbc unixodbc-dev
5 upgraded, 0 newly installed, 0 to remove and 55 not upgraded.
Need to get 714 kB of archives.
After this operation, 104 kB of additional disk space will be used.
Get:1 https://packages.m

In [6]:
# Import modules required to run demo code in this notebook
import os 
import requests
import simplejson as json
import io
import datetime
import pyodbc
import azure.cosmos.cosmos_client as cosmos_client
import azure.cosmos.errors as errors
import azure.cosmos.http_constants as http_constants


In [7]:
# Some functions to create batch ID and date/timestamp string that we will use to tag the workloads

def dtStr():
    x=datetime.datetime.now()
    dtTime=x.strftime("%Y")+"-"+x.strftime("%m")+"-"+x.strftime("%d")+" "+x.strftime("%H")+":"+x.strftime("%M")+":"+x.strftime("%S")
    return dtTime

def batchStr():
    # create a unique batchid to identify workload
    x = datetime.datetime.now()
    batchId = x.strftime("%Y%m%d%H%M%S%f")
    return batchId


In [8]:
# Job ID - unique identifier to tag each workload

batchId=batchStr()
print("Batch ID for this workload is "+batchId)

Batch ID for this workload is 20191124024716025452

In [9]:
# Set variables for Text Analytics API
# Please note, for prod implementation please include reference to Azure Key Vault instead of storing passwords/keys in plain text inside this notebook
apiEndpoint = "https://eastus.cognitiveservices.azure.com/text/analytics/v3.0-preview.1/entities/recognition/pii"
subKey = <Azure Text Analytics API key>
language = 'en'
id = 1

In [10]:
# For this demo, we have pre-configured an ADLS instance and copied a few demo files there

dataLakeAccountDNS = "Replace this with your Storage Account name" + ".blob.core.windows.net"
datalakeAccountKey = "Replace this with your Storage Key"
dataLakeConf = "fs.azure.account.key." + dataLakeAccountDNS
dataLakeExtraConfig = {dataLakeConf:datalakeAccountKey}
dataLakeContainer = "Replace this with your container name"
rawContainer = "wasbs://"+dataLakeContainer+"@" + dataLakeAccountDNS

# This is the mountpoint that will be used by Databricks to access files in ADLS 
rawMountPoint = "/mnt/raw"
dbutils.fs.mount(source = rawContainer, mount_point = rawMountPoint, extra_configs = dataLakeExtraConfig)

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-242315534026664> in <module> 
 7 rawContainer = "wasbs://adlsgen2storage@" + dataLakeAccountDNS
 8 rawMountPoint = "/mnt/raw" 
 ----> 9 dbutils . fs . mount ( source = rawContainer , mount_point = rawMountPoint , extra_configs = dataLakeExtraConfig ) 

 /local_disk0/tmp/1574563385388-0/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 312 exc . __context__ = None 
 313 exc . __cause__ = None 
 --> 314 raise exc
 315 return f_with_exception_handling
 316 

 ExecutionError : An error occurred while calling o337.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/raw; nested exception is: 
	java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/raw
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:123)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:63)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.mount(DBUtilsCore.scala:465)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/raw
	at scala.Predef$.require(Predef.scala:281)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$insertMount$1(MetadataManager.scala:205)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.withRetries(MetadataManager.scala:307)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.insertMount(MetadataManager.scala:201)
	at com.databricks.backend.daemon.data.server.handler.MountHandler.receive(MountHandler.scala:79)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1(SessionContext.scala:103)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1$adapted(SessionContext.scala:102)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at com.databricks.backend.daemon.data.server.session.SessionContext.queryHandlers(SessionContext.scala:102)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:298)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:276)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$2(ServerBackend.scala:50)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:67)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:67)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$1(ServerBackend.scala:46)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$4(UsageLogging.scala:417)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:239)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:234)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:231)
	at com.dat

In [11]:
# List files on ADLS mount 
# These are the files that we will be processing

# Full pathname - mount + directory 
inDir="/dbfs/mnt/raw/inputfiles"


In [12]:
# Azure SQLDB details. We are using this to log job details.
# For production implementation, please do not store database password in clear text.
# Note - Please add Databricks cluster IP address to firewall rules 

dbServerName = "Replace this with your instance of Azure SQL DB"
dbName = "Replace this with the database name"
dbUser = "Replace this with the database user"
dbPass = "Replace this password for database account"
driver= 'ODBC Driver 17 for SQL Server'
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+dbServerName+';PORT=1433;DATABASE='+dbName+';UID='+dbUser+';PWD='+dbPass)

In [13]:
# Azure Cosmos DB is used for storing output of Text Analytics where PII is detected.
# For this demo, we deployed an instance of Azure Cosmos DB separately. 
# Please do not store keys in clear text inside a notebook for production purposes. Please use Azure Key Vault for storing keys.

url="Replace this with Azure Cosmos DB URL"
key="Replace this with Azure Cosmos DB key"
database_id = "Replace this with Cosmos database name"
collection_id = "Replace this with Cosmos container"
database_link = 'dbs/' + database_id
collection_link = database_link + '/colls/' + collection_id
client=cosmos_client.CosmosClient(url,{'masterKey': key})

<h4> This part of the notebook loops through all the files that are stored on ADLS Gen2 directory which we mounted to Databricks cluster.</h4>
For each file processed as part of this step, Databricks creates an entry in Azure SQL DB to mark start and completion of a job. 
This step also calls Text Analytics API and captures the output of the API call and stores it into Cosmos DB if PII is detected. 

**Note on whether or not to batch input datasets that have to be parsed by Text Analytics API**<br/>
Users have option to either batch data from all the files and call Text Analytics API once or call the API once per file. 
For this demo, the code makes multiple calls to the API (once per each input file). <br/>

In [15]:


inDir="/dbfs/mnt/raw/inputfiles"
# loop through directory and append content of each file as an array element. Increment ID and keep appending records until you end up with a single Json document
for file in os.listdir(inDir):
    # initialise id counter
    strObj = " "
    
    # read contents of the file and store it in a variable
    fullFileName=inDir+"/"+file 
    readFile = open(inDir+"/"+file,"r")
    
    print("Processing file .. "+fullFileName)

    # Insert job status in Azure SQL DB when file processing starts
    # writing to azure sqldb
    stDtTime=dtStr()
    insertStr="insert into dbo.joblogs values ('"+batchId+"','"+fullFileName+"','Started','"+stDtTime+"')"
    cursor = cnxn.cursor()
    cursor.execute(insertStr)
    cnxn.commit()

    for line in readFile:
        
        # read lines into a string object 
        strObj = strObj + line

    # create JSON structure that can be processed by Text Analytics API
    jsonDoc = { "documents": [{"language":language,"id":id,"Text":strObj}]}

    # call API 
    headers = {"Ocp-Apim-Subscription-Key": subKey}
    response = requests.post(apiEndpoint, headers=headers, json=jsonDoc)
    entities = response.json()
 
    # print(json.dumps(entities, separators=(',', ':'), sort_keys=True, indent = 4 * ' '))

    # If PII is detected by the API - write output to Cosmos DB container else ignore the entry
    if len(entities['documents'][0]['entities']) != 0:

        print("potential pii present")

        # Write API output to Cosmos DB collection using SQL API
        print('Writing API results to Cosmos DB ... ')
        # Add batch ID and filename to the JSON document before it gets written to Cosmos DB
        entities.__setitem__('batchid',batchId)
        entities.__setitem__('input data',fullFileName)
        
        # Write document to Cosmos DB collection
        client.CreateItem(collection_link,entities)
        
        # Insert job status into Azure SQL DB
        stDtTime=dtStr()
        insertStr="insert into dbo.joblogs values ('"+batchId+"','"+fullFileName+"','Completed','"+stDtTime+"')"
        cursor = cnxn.cursor()
        cursor.execute(insertStr)
        cnxn.commit()
  
    else:
        # Insert job status into Azure SQL DB
        stDtTime=dtStr()
        insertStr="insert into dbo.joblogs values ('"+batchId+"','"+fullFileName+"','Completed','"+stDtTime+"')"
        cursor = cnxn.cursor()
        cursor.execute(insertStr)
        cnxn.commit()

Processing file .. /dbfs/mnt/raw/inputfiles/demo.txt
potential pii present
Writing API results to Cosmos DB ... 
Processing file .. /dbfs/mnt/raw/inputfiles/file1.txt
potential pii present
Writing API results to Cosmos DB ... 
Processing file .. /dbfs/mnt/raw/inputfiles/file2.txt
potential pii present
Writing API results to Cosmos DB ... 
Processing file .. /dbfs/mnt/raw/inputfiles/file3.txt
potential pii present
Writing API results to Cosmos DB ... 
Processing file .. /dbfs/mnt/raw/inputfiles/file4.txt
potential pii present
Writing API results to Cosmos DB ... 
Processing file .. /dbfs/mnt/raw/inputfiles/input.json
potential pii present
Writing API results to Cosmos DB ... 
Processing file .. /dbfs/mnt/raw/inputfiles/pii.txt
potential pii present
Writing API results to Cosmos DB ... 
Processing file .. /dbfs/mnt/raw/inputfiles/testfile.txt
potential pii present
Writing API results to Cosmos DB ...